In [1]:
# Simulation of a single 20-bead linear chain

import copy
import feasst as fst
print(fst.version())

def end_to_end_distance(mc):
    config = mc.system().configuration()
    pos = config.particle(0).site(0).position()
    return pos.distance(config.particle(0).site(config.particle(0).num_sites() - 1).position())

mc = fst.MonteCarlo()
mc.set(fst.MakeRandomMT19937(fst.args({"seed": "time"})))
mc.add(fst.Configuration(fst.MakeDomain(fst.args({"cubic_box_length": "30"})),
                         fst.args({"particle_type": fst.install_dir() + "/plugin/chain/forcefield/data.chain20"})))
mc.add(fst.MakePotential(fst.MakeLennardJones(),
                         fst.MakeVisitModelIntra(fst.args({"cutoff": "1"}))))
mc.set(fst.MakeThermoParams(fst.args({"beta": "1", "chemical_potential0": "1"})))
mc.set(fst.MakeMetropolis())
fst.SeekNumParticles(1).with_trial_add().run(mc)
mc.add(fst.MakeTrialPivot(fst.args({"weight": "1", "tunable_param": "20", "max_length": "8"})))
mc.add(fst.MakeTrialCrankshaft(fst.args({"weight": "1", "tunable_param": "20", "max_length": "8"})))
mc.add(fst.MakeTrialReptate(fst.args({"weight": "1", "max_length": "1"})))
unit_grows = list()
grows=list()
length=mc.configuration().num_sites()
max_length=19
for i in range(max_length):
     unit_grows.append([{"bond": "true", "mobile_site": str(i), "anchor_site": str(i+1)}])
     if i == 0:
         grows.append(unit_grows[-1])
     else:
         grows.append(unit_grows[-1] + copy.deepcopy(grows[-1]))
unit_grows = list()
for i in range(max_length):
    unit_grows.append([{"bond": "true", "mobile_site": str(length-i-1), "anchor_site": str(length-i-2)}])
    if i == 0:
        grows.append(unit_grows[-1])
    else:
        grows.append(unit_grows[-1] + copy.deepcopy(grows[-1]))
for grow in grows:
    grow[0]["weight"] = str(1./len(grow))
    grow[0]["particle_type"] = "0"
    mc.add(fst.MakeTrialGrow(fst.ArgsVector(grow), fst.args({"num_steps": "4"})))
steps_per = str(int(1e4))
mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": steps_per, "file_name": "chain"})))
mc.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per": steps_per, "tolerance": str(1e-10)})))
mc.add(fst.MakeCheckRigidBonds(fst.args({"steps_per": steps_per})))
print('Potential energy of initial configuration:', mc.criteria().current_energy())
rg_sq = fst.Accumulator()
print('end to end', end_to_end_distance(mc))
mc.attempt(int(1e3)) # equilibrate
for n in range(int(1e5)):
    mc.attempt()
    rg_sq.accumulate(end_to_end_distance(mc))

print(rg_sq.str())

v0.16.0-6-gb9ee4eb228-dirty-hwh/jacob
[{'bond': 'true', 'mobile_site': '0', 'anchor_site': '1', 'weight': '1.0', 'particle_type': '0'}]
[{'bond': 'true', 'mobile_site': '1', 'anchor_site': '2', 'weight': '0.5', 'particle_type': '0'}, {'bond': 'true', 'mobile_site': '0', 'anchor_site': '1'}]
[{'bond': 'true', 'mobile_site': '2', 'anchor_site': '3', 'weight': '0.3333333333333333', 'particle_type': '0'}, {'bond': 'true', 'mobile_site': '1', 'anchor_site': '2'}, {'bond': 'true', 'mobile_site': '0', 'anchor_site': '1'}]
[{'bond': 'true', 'mobile_site': '3', 'anchor_site': '4', 'weight': '0.25', 'particle_type': '0'}, {'bond': 'true', 'mobile_site': '2', 'anchor_site': '3'}, {'bond': 'true', 'mobile_site': '1', 'anchor_site': '2'}, {'bond': 'true', 'mobile_site': '0', 'anchor_site': '1'}]
[{'bond': 'true', 'mobile_site': '4', 'anchor_site': '5', 'weight': '0.2', 'particle_type': '0'}, {'bond': 'true', 'mobile_site': '3', 'anchor_site': '4'}, {'bond': 'true', 'mobile_site': '2', 'anchor_site'

average,stdev,block_stdev,n,moment0,moment1,
2.9065817916105852,1.0735913277316973,0,100000,290658.1791610585,960080.45243195181,
